# Homework 4

**ÖNEMLİ:** Beyler aşağıda her bölümde yapılması gereken seyleri belirttim. Herkes tamamladığı kısımların üstünü çift tilda (\~~) ile çizerse geriye ne kalmış anlayabiliriz. Ayrıca **OutputTable** adında bir class olusturdum, output table olusturmak icin uzun uzun yazmanıza gerek yok kullanımına BFGS kodundan bakabilirsiniz.

**ÇOK ÖNEMLİ:** Beyler np.array kullanarak listten array olusturdugunuzda shape i yanlıs geliyor ve matrix gibi algılamayıp islemlerde mal oluyor. Bu yüzden array olusturdugunuzda (özellikle x0 ve grad_f icin) np.reshape kullanarak düzgün shape'i vermeyi unutmayın.

Yazıyı okuyanlar isminin üstünü cizebilirse herkes okuyunca bu yazıyı silebiliriz:
*   ~~Sefa~~
*   ~~Yunus~~
*   Harun
*   ~~Akın~~

### TODOs

Steepest Descent:
*   Algorithm code
*   ~~Function 1 - Solution set 1~~
*   ~~Function 1 - Solution set 2~~
*   ~~Function 2 - Solution set 1~~
*   ~~Function 2 - Solution set 2~~

Newton's Method:
*   ~~Algorithm code~~
*   ~~Function 1 - Solution set 1~~
*   ~~Function 1 - Solution set 2~~
*   ~~Function 2 - Solution set 1~~
*   ~~Function 2 - Solution set 2~~

DFP:
*   Algorithm code
*   Function 1 - Solution set 1
*   Function 1 - Solution set 2
*   Function 2 - Solution set 1
*   Function 2 - Solution set 2  

BFGS:
*   ~~Algorithm code~~
*   ~~Function 1 - Solution set 1~~
*   Function 1 - Solution set 2
*   Function 2 - Solution set 1
*   Function 2 - Solution set 2

In [3]:
import pandas as pd
import numpy as np
from sympy import Symbol, lambdify

In [4]:
x1 = Symbol("x1")
x2 = Symbol("x2")

func1 = (5*x1 - x2)**4 + (x1 - 2)**2 + x1 - 2*x2 + 12
func2 = 100*(x2 - x1**2)**2 + (1 - x1)**2 


f1 = lambdify([[x1,x2]], func1, "numpy")
f2 = lambdify([[x1,x2]], func2, "numpy")

gf1 = lambdify([[x1,x2]], func1.diff([[x1, x2]]), "numpy")
gf2 = lambdify([[x1,x2]], func2.diff([[x1, x2]]), "numpy")

grad_f1 = lambda x_arr : np.array(gf1(x_arr)).reshape(1,2)
grad_f2 = lambda x_arr : np.array(gf2(x_arr)).reshape(1,2)

hf1 = lambdify([[x1,x2]], (func1.diff([[x1, x2]])).diff([[x1, x2]]), "numpy")
hf2 = lambdify([[x1,x2]], (func2.diff([[x1, x2]])).diff([[x1, x2]]), "numpy")

hess_f1= lambda x_arr : np.array(hf1(np.array(x_arr).reshape(2,)))
hess_f2= lambda x_arr : np.array(hf2(np.array(x_arr).reshape(2,)))

In [5]:
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

# plot the function
x = np.arange(0,3,0.01)
y = np.arange(0,3,0.01)
X,Y = meshgrid(x, y) # grid of point
Z = f2([X,Y]) # evaluation of the function on the grid

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, 
                      cmap=cm.RdBu,linewidth=0, antialiased=False)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.savefig("graph.png")
plt.show()

<Figure size 640x480 with 2 Axes>

### Useful Functions

In [7]:
np_str = lambda x_k : np.array2string(x_k.reshape(len(x_k)), precision=3, separator=',')

f_str = lambda x : "{0:.4f}".format(x)

In [8]:
class OutputTable:    
    def __init__(self):
        self.table = pd.DataFrame([],columns=['k', 'x^k', 'f(x^k)', 'd^k', 'a^k', 'x^k+1'])
    def add_row(self, k, xk, fxk, dk, ak, xkp):
        self.table.loc[len(self.table)] = [k, np_str(xk), f_str(np.asscalar(fxk)), np_str(dk), ak, np_str(xkp)]
    def print_latex(self):
        print(self.table.to_latex(index=False))

### Exact Line Search

In [11]:
def BisectionMethod(f, a=-100,b=100,epsilon=0.0000000001) :
    iteration=0
    while (b - a) >= epsilon:
        x_1 = (a + b) / 2
        fx_1 = f(x_1)
        if f(x_1 + epsilon) <= fx_1:
            a = x_1
        else:
            b = x_1
        iteration+=1
    x_star = (a+b)/2
    return x_star

def ExactLineSearch(f, x0, d, eps=0.0000000001):
    alpha = Symbol('alpha')
    function_alpha = f(np.array(x0)+alpha*np.array(d))
    f_alp = lambdify(alpha, function_alpha, 'numpy')
    alp_star = BisectionMethod(f_alp, epsilon=eps)
    return alp_star

## Steepest Descent Method

In [ ]:
def steepestDescentMethod(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    stop = False
    output = OutputTable()
    while(stop == False):
        d = - np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        else:
            a = ExactLineSearch(f,xk,d)
            xkp = xk + a*d
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output

In [ ]:
xs1, fs1, outputs1 = steepestDescentMethod(f1, grad_f1, [10,10], 0.001)
xs1, fs1

In [ ]:
print(outputs1.table.to_latex(index=False))

In [ ]:
xs2, fs2, outputs2 = steepestDescentMethod(f1, grad_f1, [-25,-15], 0.001)
xs2, fs2

In [ ]:
print(outputs2.table.to_latex(index=False))

In [ ]:
xs3, fs3, outputs3 = steepestDescentMethod(f2, grad_f2, [2,-4], 0.01)
xs3, fs3

In [ ]:
print(outputs3.table.to_latex(index=False))

In [ ]:
xs4, fs4, outputs4 = steepestDescentMethod(f2, grad_f2, [-2,-3.5], 0.002)
xs4, fs4

In [ ]:
outputs4.table

## Newton's Method

In [12]:
def NewtonsMethod(x_0,epsilon,f,grad_f,Hessian_f):
    xk = np.array(x_0).reshape(2,1)
    k=0
    output = OutputTable()
    while(True):
        d_k=-np.linalg.inv(Hessian_f(xk))@np.transpose(grad_f(xk))
        alpha_k=ExactLineSearch(f,xk,d_k)
        xkp=xk+alpha_k*d_k
        if(np.linalg.norm(grad_f(xk)) < epsilon):
            break
        output.add_row(k, xk, f(xk), d_k, alpha_k, xkp)
        xk = xkp
        k += 1
    output.add_row(k,xk,f(xk),d_k,None,np.array([]))    
    return xk, np.asscalar(f(xk)), output

def NewtonsMethod_vol2(x_0,epsilon,f,grad_f,Hessian_f):
    xk = x_0
    k=0
    H = np.identity(len(x_0))
    output = OutputTable()
    while(True):
        #print(xk)
        a=np.asarray(Hessian_f(xk))
        #print(a)
        #d_k=-np.linalg.inv(a)@np.transpose(np.asarray(grad_f(xk)).reshape(1,2))
        d_k=-np.linalg.inv(a)@np.transpose(np.asarray(gf2(xk)).reshape(1,2))
        alpha_k=ExactLineSearch(f,np.array(xk).reshape(2,1),d_k)
        xkp=[x + y for x, y in zip(x_0,(alpha_k*d_k).flatten().tolist() )]
        #print(xkp)
        print(np.linalg.norm(np.asarray(grad_f(xk)).reshape(1,2)))
        if(np.linalg.norm(np.asarray(grad_f(xk)).reshape(1,2)) < epsilon):
            break
        #output.add_row(k, np.array(xk), f(np.array(xk)), d_k, alpha_k, np.array(xkp))
        xk= xkp
        k += 1
        
    #output.add_row(k,np.array(xk),f(np.array(xk)),d_k,None,np.array([]))    
    return (xk)



In [13]:
x_f1_s1,f1_s1, outputf1_1 = NewtonsMethod([0,1], 0.001,f1,grad_f1,hess_f1)
outputf1_1.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,1]",15.0000,"[ 6.5 ,32.333]",1.000620,"[ 6.504,33.353]"
1,1,"[ 6.504,33.353]",-27.4344,"[-0.004,-0.058]",1.048271,"[ 6.5 ,33.293]"
2,2,"[ 6.5 ,33.293]",-27.4406,"[0. ,0.001]",-10.547173,"[ 6.498,33.283]"
3,3,"[ 6.498,33.283]",-27.4405,"[0.002,0.011]",2.118884,"[ 6.503,33.306]"
4,4,"[ 6.503,33.306]",-27.4405,"[-0.003,-0.012]",2.107937,"[ 6.497,33.28 ]"
5,5,"[ 6.497,33.28 ]",-27.4405,"[0.003,0.014]",2.598188,"[ 6.504,33.315]"
6,6,"[ 6.504,33.315]",-27.4405,"[-0.004,-0.022]",0.390601,"[ 6.503,33.307]"
7,7,"[ 6.503,33.307]",-27.4405,"[-0.003,-0.013]",3.190661,"[ 6.494,33.265]"
8,8,"[ 6.494,33.265]",-27.4405,"[0.006,0.029]",1.626592,"[ 6.504,33.312]"
9,9,"[ 6.504,33.312]",-27.4405,"[-0.004,-0.018]",1.457198,"[ 6.498,33.285]"


In [24]:
x_f1_s2,f1_s2, outputf1_2 = NewtonsMethod([-25,75], 0.0001,f1,grad_f1,hess_f1)
outputf1_2.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[-25, 75]",1600000566.0000,"[31.5 ,90.833]",2.962919,"[ 68.332,344.132]"
1,1,"[ 68.332,344.132]",3829.3422,"[ -61.832,-309.956]",1.001747,"[ 6.392,33.634]"
2,2,"[ 6.392,33.634]",-21.7344,"[0.108,0.042]",1.756464,"[ 6.582,33.707]"
3,3,"[ 6.582,33.707]",-27.4338,"[-0.082,-0.413]",1.002057,"[ 6.5 ,33.293]"
4,4,"[ 6.5 ,33.293]",-27.4406,"[0. ,0.001]",49.983209,"[ 6.508,33.334]"
...,...,...,...,...,...,...
277,277,"[ 6.5 ,33.293]",-27.4406,"[0. ,0.001]",99.218652,"[ 6.514,33.364]"
278,278,"[ 6.514,33.364]",-27.4404,"[-0.014,-0.07 ]",1.020570,"[ 6.5 ,33.292]"
279,279,"[ 6.5 ,33.292]",-27.4406,"[0. ,0.001]",-63.115389,"[ 6.481,33.201]"
280,280,"[ 6.481,33.201]",-27.4402,"[0.019,0.093]",1.002097,"[ 6.5 ,33.294]"


In [26]:
x_f2_s1,f2_s1, outputf2_1=NewtonsMethod([10,22], 0.0001,f2,grad_f2,hess_f2)
outputf2_1.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[10,22]",608481.0000,"[-5.769e-04, 7.799e+01]",1.000000,"[ 9.999,99.988]"
1,1,"[ 9.999,99.988]",80.9896,"[ -9.001,-180.002]",0.038997,"[ 9.648,92.969]"
2,2,"[ 9.648,92.969]",76.3131,"[-0.337,-6.386]",1.313111,"[ 9.206,84.584]"
3,3,"[ 9.206,84.584]",69.8143,"[-0.252,-4.488]",2.337646,"[ 8.616,74.093]"
4,4,"[ 8.616,74.093]",59.8769,"[-0.268,-4.482]",1.913713,"[ 8.103,65.515]"
5,5,"[ 8.103,65.515]",52.3520,"[-0.248,-3.887]",2.157086,"[ 7.567,57.131]"
6,6,"[ 7.567,57.131]",44.7463,"[-0.248,-3.627]",2.031330,"[ 7.063,49.763]"
7,7,"[ 7.063,49.763]",38.2633,"[-0.238,-3.233]",2.106886,"[ 6.563,42.952]"
8,8,"[ 6.563,42.952]",32.2582,"[-0.232,-2.934]",2.071382,"[ 6.081,36.875]"
9,9,"[ 6.081,36.875]",26.9983,"[-0.224,-2.614]",2.099012,"[ 5.612,31.389]"


In [23]:
x_f2_s2,f2_s2, outputf2_2=NewtonsMethod([-25,35], 0.001,f2,grad_f2,hess_f2)
outputf2_2.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[-25, 35]",34810676.0000,"[2.203e-04,5.900e+02]",1.000000,"[-25. ,624.989]"
1,1,"[-25. ,624.989]",675.9885,"[ 26. ,-1300.003]",0.019357,"[-24.496,599.824]"
2,2,"[-24.496,599.824]",656.4871,"[ 0.494,-23.926]",1.283239,"[-23.863,569.121]"
3,3,"[-23.863,569.121]",629.0237,"[ 0.372,-17.416]",2.280809,"[-23.015,529.398]"
4,4,"[-23.015,529.398]",585.5685,"[ 0.397,-17.982]",1.907868,"[-22.257,495.091]"
5,5,"[-22.257,495.091]",550.1546,"[ 0.376,-16.446]",2.101430,"[-21.467,460.531]"
6,6,"[-21.467,460.531]",513.1846,"[ 0.38,-16.04]",1.995883,"[-20.708,428.517]"
7,7,"[-20.708,428.517]",479.4674,"[ 0.371,-15.097]",2.051616,"[-19.945,397.544]"
8,8,"[-19.945,397.544]",446.4869,"[ 0.369,-14.441]",2.025496,"[-19.198,368.293]"
9,9,"[-19.198,368.293]",415.3993,"[ 0.364,-13.69 ]",2.037811,"[-18.457,340.396]"


## DFP

In [28]:
def DFP(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    H = np.identity(len(x_0))
    stop = False
    output = OutputTable()
    while(stop == False):
        d = -H @ np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        else:
            a = ExactLineSearch(f,xk,d)
            xkp = xk + a*d
            p = xkp - xk
            q = np.transpose(grad_f(xkp)) - np.transpose(grad_f(xk))
            A = (p @ np.transpose(p)) / (p.transpose() @ q)
            B = - (H @ q @ np.transpose( H @ q)) / (q.transpose() @ H @ q)
            Hkp = H + A + B
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
            H = Hkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output

In [29]:
xs1, fs1, output1 = DFP(f1, grad_f1, [0,0], 0.001)
xs1, fs1

(array([[ 6.50004193],
        [33.29392539]]), -27.44055078634222)

In [30]:
output1.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,0]",16,"[3.,2.]",0.0442505,"[0.133,0.089]"
1,1,"[0.133,0.089]",15.5519,"[0.491,2.493]",13.1149,"[ 6.575,32.79 ]"
2,2,"[ 6.575,32.79 ]",-26.0735,"[-0.057, 0.706]",0.883484,"[ 6.524,33.414]"
3,3,"[ 6.524,33.414]",-27.44,"[-0.024,-0.117]",0.898743,"[ 6.502,33.309]"
4,4,"[ 6.502,33.309]",-27.4405,"[-0.006,-0.038]",0.395203,"[ 6.5 ,33.294]"
5,5,"[ 6.5 ,33.294]",-27.4406,"[-4.193e-05,-2.248e-04]",None,[]


Akin kardesime selam olsun. 4. Katta bulusmak uzere...
                                Sevgiler
                                 Harun

## BFGS

In [193]:
def BFGS(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    H = np.identity(len(x_0))
    stop = False
    output = OutputTable()
    while(stop == False):
        d = -H @ np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        if(k == -1):
            break
        else:
            a = ExactLineSearch(f,xk,d, eps=0.00001)
            xkp = xk + a*d
            p = xkp - xk
            q = np.transpose(grad_f(xkp)) - np.transpose(grad_f(xk))
            A = ((1+ np.transpose(q) @ H @ q) / (np.transpose(q) @ p)) * (p @ np.transpose(p)) / (np.transpose(p) @ q)
            B = - (p @ np.transpose(q) @ H + H @ q @ np.transpose(p)) / (np.transpose(q) @ p)
            Hkp = H + A + B
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
            H = Hkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output


In [194]:
xs1, fs1, output1 = BFGS(f1, grad_f1, [0,0], 0.001)
xs1, fs1

(array([[ 6.49612176],
        [33.27399017]]), -27.44053536334784)

In [23]:
output1.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,0]",16,"[3.,2.]",0.0442505,"[0.133,0.089]"
1,1,"[0.133,0.089]",15.5519,"[0.963,4.868]",6.66656,"[ 6.553,32.54 ]"
2,2,"[ 6.553,32.54 ]",-25.7982,"[-0.173, 2.234]",0.325012,"[ 6.496,33.266]"
3,3,"[ 6.496,33.266]",-27.4402,"[-0.001, 0.014]",0.520325,"[ 6.496,33.273]"
4,4,"[ 6.496,33.273]",-27.4405,"[-0.005, 0.067]",-0.00152588,"[ 6.496,33.273]"
5,5,"[ 6.496,33.273]",-27.4405,"[ -71.486,1029.42 ]",-0.00152588,"[ 6.605,31.702]"
6,6,"[ 6.605,31.702]",-20.5324,"[-0.005, 0.066]",23.8388,"[ 6.497,33.279]"
7,7,"[ 6.497,33.279]",-27.4405,"[3.545e-05,1.858e-04]",71.0648,"[ 6.5 ,33.292]"
8,8,"[ 6.5 ,33.292]",-27.4405,"[3.545e-05,1.858e-04]",None,[]


In [24]:
xs2, fs2, output2 = BFGS(f1, grad_f1, [10,10], 0.001)
xs2, fs2

(array([[ -615682.95451531],
        [-3077907.80209916]]), 445132303112.5189)

In [25]:
output2.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[10,10]",2560066,"[-1280017., 256002.]",-0.00152588,"[1963.151,-380.628]"
1,1,"[1963.151,-380.628]",1.0809e+16,"[-0.27 ,-1.346]",99.9985,"[1936.113,-515.196]"
2,2,"[1936.113,-515.196]",1.08063e+16,"[11204.828,56024.005]",-99.9985,"[-1118529.603,-5602830.199]"
3,3,"[-1118529.603,-5602830.199]",1.07502e+16,"[152481.149,762452.782]",99.9985,"[14129352.638,70641284.582]"
4,4,"[14129352.638,70641284.582]",1.10055e+15,"[ -4975.245,-24876.173]",99.9985,"[13631835.74 ,68153705.264]"
5,5,"[13631835.74 ,68153705.264]",1.08334e+15,"[15.742,78.718]",99.9985,"[13633409.883,68161576.963]"
6,6,"[13633409.883,68161576.963]",1.08274e+15,"[-0.203,-1.016]",99.9985,"[13633389.563,68161475.364]"
7,7,"[13633389.563,68161475.364]",1.08273e+15,"[ -20618.895,-103093.78 ]",99.9985,"[11571531.546,57852254.692]"
8,8,"[11571531.546,57852254.692]",9.86121e+14,"[0.029,0.147]",99.9985,"[11571534.465,57852269.344]"
9,9,"[11571534.465,57852269.344]",9.86086e+14,"[0.077,0.389]",99.9985,"[11571542.211,57852308.238]"


In [238]:
xs3, fs3, output3 = BFGS(f2, grad_f2, [0,0], 0.001)
xs3, fs3

(array([[ 10.3087146 ],
        [106.28439447]]), 86.67406483533888)

In [239]:
output3.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,0]",1,"[2.,0.]",0.0778198,"[0.156,0. ]"
1,1,"[0.156,0. ]",0.771623,"[14.237, 5.326]",0.00762939,"[0.264,0.041]"
2,2,"[0.264,0.041]",0.626576,"[0.466,0.438]",0.779724,"[0.627,0.382]"
3,3,"[0.627,0.382]",0.151823,"[-0.36 , 0.052]",3.83759,"[-0.754, 0.582]"
4,4,"[-0.754, 0.582]",3.09615,"[-0.967,-0.186]",0.00152588,"[-0.755, 0.582]"
5,5,"[-0.755, 0.582]",3.09466,"[-0.777,-0.211]",-2.3056,"[1.036,1.069]"
6,6,"[1.036,1.069]",0.00262663,"[-4.95 ,-1.609]",-0.00152588,"[1.043,1.071]"
7,7,"[1.043,1.071]",0.0305442,"[-0.275,-0.09 ]",6.32477,"[-0.699, 0.499]"
8,8,"[-0.699, 0.499]",2.89796,"[-0.065,-0.087]",0.0198364,"[-0.701, 0.497]"
9,9,"[-0.701, 0.497]",2.89633,"[-0.068,-0.127]",0.00457764,"[-0.701, 0.497]"


In [240]:
xs4, fs4, output4 = BFGS(f2, grad_f2, [5,5], 0.001)
xs4, fs4

(array([[66.26186956],
        [27.35226086]]), 1903828197.1174762)

In [241]:
output4.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[5,5]",40016,"[-40008., 4000.]",-0.00152588,"[66.047,-1.104]"
1,1,"[66.047,-1.104]",1.90389e+09,"[ 28.116,3729.756]",0.00762939,"[66.262,27.352]"
2,2,"[66.262,27.352]",1.90383e+09,"[8.292e-07,1.108e-04]",None,[]


In [345]:
from scipy.optimize import minimize

In [354]:
minimize(fun=f1, x0=[0,0], method='BFGS', tol=0.005)


      fun: -27.440550788387824
 hess_inv: array([[ 0.47354836,  2.36296605],
       [ 2.36296605, 11.92423175]])
      jac: array([-3.86714935e-04,  7.15255737e-05])
  message: 'Optimization terminated successfully.'
     nfev: 64
      nit: 11
     njev: 16
   status: 0
  success: True
        x: array([ 6.49998414, 33.29363065])